In [11]:
import pandas as pd

In [12]:
dataset_agg_question_mean = (
    pd.read_parquet("./data/dataset.agg.question.parquet")
    .filter(like="mean")
    .droplevel(axis=1, level=None)
)
dataset_agg_question_mean

model         gemma:7b                                  llama2:70b  \
persona           base conservative   liberal  moderate       base   
segment   id                                                         
agreement 1   5.000000     1.680000  4.960000  3.000000       4.02   
          2   5.000000     3.240000  5.000000  4.380000       4.00   
          3   5.000000     4.900000  3.440000  3.000000       4.00   
          4   4.960000     3.340000  3.320000  3.080000       4.00   
          5   4.880000     5.000000  5.000000  4.220000       3.40   
          6   5.000000     5.000000  5.000000  5.000000       5.00   
          7   5.000000     5.000000  5.000000  5.000000       4.14   
          8   5.000000     2.740000  4.920000  3.160000       4.00   
          9   4.660000     2.380000  2.000000  2.140000       4.00   
          10  2.000000     2.000000  2.000000  2.000000       1.18   
          11  3.600000     2.140000  2.000000  2.140000       1.78   
          12  5.000000     5.000000  5.000000  5.000000       4.86   
          13  4.560000     4.260000  5.000000  3.980000       3.82   
          14  2.040000     2.080000  2.040000  2.040000       3.68   
          15  3.340000     2.280000  2.000000  2.040000       3.28   
          16  5.000000     3.940000  3.420000  3.080000       4.00   
relevance 1   3.000000     3.000000  3.000000  3.000000       2.82   
          2   3.020000     3.000000  3.023810  3.000000       3.12   
          3   4.020408     3.938776  3.625000  3.155556       3.76   
          4   3.560000     2.979167  3.000000  3.000000       3.66   
          5   4.020000     3.000000  2.977273  3.000000       3.86   
          6   2.918367     1.000000  1.000000  0.979592       1.12   
          7   3.980000     3.208333  3.666667  3.020408       3.84   
          8   4.063830     2.979167  3.040816  3.000000       3.94   
          9   4.020000     3.612245  3.733333  3.062500       3.90   
          10  3.000000     2.980000  3.000000  3.000000       3.20   
          11  4.000000     3.065217  3.063830  3.000000       3.44   
          12  4.000000     3.000000  3.000000  3.000000       3.86   
          13  4.020000     3.959184  4.000000  3.976744       3.96   
          14  3.041667     3.000000  3.000000  3.000000       3.50   
          15  3.360000     2.954545  3.023256  3.000000       3.44   
          16  4.000000     2.977778  2.955556  3.000000       4.00   

model                                      llama3:70b               ...  \
persona      conservative liberal moderate       base conservative  ...   
segment   id                                                        ...   
agreement 1      3.780000    4.22     4.00       5.00         3.00  ...   
          2      3.775510    4.00     4.00       5.00         4.72  ...   
          3      4.145833    3.42     3.34       4.88         5.00  ...   
          4      4.000000    2.08     3.80       4.86         5.00  ...   
          5      3.400000    2.30     4.00       4.06         5.00  ...   
          6      4.081633    4.64     4.50       5.00         5.00  ...   
          7      3.877551    4.02     4.00       5.00         5.00  ...   
          8      3.530612    4.12     4.00       5.00         5.00  ...   
          9      4.000000    2.94     3.30       2.78         5.00  ...   
          10     1.000000    1.00     1.12       2.00         5.00  ...   
          11     1.720000    1.00     2.40       2.26         5.00  ...   
          12     4.163265    2.18     4.00       5.00         5.00  ...   
          13     1.125000    3.88     3.14       5.00         1.74  ...   
          14     3.632653    2.32     2.78       2.00         5.00  ...   
          15     2.620000    1.00     3.46       4.78         5.00  ...   
          16     3.938776    1.12     2.76       4.12         5.00  ...   
relevance 1      1.897959    2.08     2.02       4.00         3.34  ...   
          2      2.816327    3.32     2.96       

In [13]:
dataset_agg_question_var = (
    pd.read_parquet("./data/dataset.agg.question.parquet")
    .filter(like="var")
    .droplevel(axis=1, level=None)
)
dataset_agg_question_var

model         gemma:7b                                  llama2:70b  \
persona           base conservative   liberal  moderate       base   
segment   id                                                         
agreement 1   0.000000     0.222041  0.039184  0.244898   0.020000   
          2   0.000000     1.206531  0.000000  0.240408   0.000000   
          3   0.000000     0.091837  0.292245  0.000000   0.000000   
          4   0.039184     0.351429  0.344490  0.075102   0.000000   
          5   0.107755     0.000000  0.000000  0.175102   1.469388   
          6   0.000000     0.000000  0.000000  0.000000   0.000000   
          7   0.000000     0.000000  0.000000  0.000000   0.122857   
          8   0.000000     0.196327  0.075102  0.137143   0.000000   
          9   0.228980     0.485306  0.000000  0.245306   0.000000   
          10  0.000000     0.000000  0.000000  0.000000   0.517959   
          11  0.244898     0.122857  0.000000  0.245306   1.766939   
          12  0.000000     0.000000  0.000000  0.000000   0.122857   
          13  0.251429     0.318776  0.000000  0.020000   0.517959   
          14  0.039184     0.075102  0.039184  0.039184   0.222041   
          15  0.637143     0.205714  0.000000  0.080000   1.675102   
          16  0.000000     0.261633  0.289388  0.075102   0.000000   
relevance 1   0.000000     0.000000  0.000000  0.000000   0.599592   
          2   0.020000     0.000000  0.023810  0.000000   0.107755   
          3   0.020408     0.058673  0.239362  0.134343   0.186122   
          4   0.251429     0.020833  0.000000  0.000000   0.228980   
          5   0.020000     0.000000  0.022727  0.000000   0.122857   
          6   0.076531     0.000000  0.000000  0.020408   0.107755   
          7   0.020000     0.168440  0.226950  0.020408   0.137143   
          8   0.061055     0.020833  0.039966  0.000000   0.057551   
          9   0.020000     0.242347  0.200000  0.059840   0.091837   
          10  0.000000     0.020000  0.000000  0.000000   0.163265   
          11  0.000000     0.062319  0.061055  0.000000   0.986122   
          12  0.000000     0.000000  0.000000  0.000000   0.122857   
          13  0.020000     0.081633  0.000000  0.023256   0.039184   
          14  0.040780     0.000000  0.000000  0.000000   0.255102   
          15  0.235102     0.044397  0.023256  0.000000   0.251429   
          16  0.000000     0.022222  0.043434  0.000000   0.000000   

model                                         llama3:70b               ...  \
persona      conservative   liberal  moderate       base conservative  ...   
segment   id                                                           ...   
agreement 1      0.215918  0.175102  0.000000   0.000000     0.000000  ...   
          2      0.427721  0.000000  0.000000   0.000000     0.205714  ...   
          3      0.127216  0.575102  0.228980   0.107755     0.000000  ...   
          4      0.000000  1.136327  0.408163   0.122857     0.000000  ...   
          5      1.469388  2.214286  0.000000   0.098367     0.000000  ...   
          6      0.076531  0.235102  0.255102   0.000000     0.000000  ...   
          7      0.359694  0.020000  0.000000   0.000000     0.000000  ...   
          8      1.129252  0.107755  0.000000   0.000000     0.000000  ...   
          9      0.000000  0.833061  0.622449   0.664898     0.000000  ...   
          10     0.000000  0.000000  0.230204   0.000000     0.000000  ...   
          11     1.675102  0.000000  2.000000   0.400408     0.000000  ...   
          12     0.139456  0.762857  0.000000   0.000000     0.000000  ...   
          13     0.367021  0.597551  1.428980   0.000000     0.196327  ...   
          14     0.987245  0.711837  0.542449   0.000000     0.000000  ...   
          15     2.281224  0.000000  1.355510   0.175102     0.000000  ...   
          16     0.183673  0.230204  1.614694   0.107755     0.000000  ...   
relevance 1      0.176871  0.075102  0.101633   0.000000     0.5555

In [14]:
(
    dataset_agg_question_var.mean()
    .reset_index()
    .pivot(index="model", columns="persona")
    .T.droplevel(level=0)
    .T.pipe(lambda _df: _df.assign(MEAN=_df.mean(axis=1)))
    .T.pipe(lambda _df: _df.assign(MEAN=_df.mean(axis=1)))
    .T
    # .to_latex(
    #     buf=open(f"tab.questionnaire.var.agg.tex", "w"),
    #     label=f"tab:questionnaire:questionnaire.var.agg",
    #     escape=True,
    #     float_format="{:0.3f}".format
    # )
)

persona,base,conservative,liberal,moderate,MEAN
model,,,,,
gemma:7b,0.072934,0.133727,0.061255,0.057369,0.081321
llama2:70b,0.309145,0.513819,0.421696,0.447079,0.422935
llama3:70b,0.115587,0.061633,0.088801,0.299554,0.141393
mistral:7b,0.259016,0.665350,0.203553,0.488868,0.404197
mixtral:8x22b,0.161990,0.133648,0.112130,0.179643,0.146853
mixtral:8x7b,0.024785,0.037156,0.047231,0.012117,0.030322
qwen:72b,0.108087,0.115712,0.356110,1.122053,0.425490
MEAN,0.150221,0.237292,0.184397,0.372383,0.236073
